# 1. Setup

In [1]:
import os
import time
import pandas as pd
import json
import requests
from binance.client import Client

# 2. Authenticate

In [2]:
# Khóa API của Binance
apikey = '...'
secret = '...'

# Khởi tạo client Binance
client = Client(apikey, secret)

# API key của CoinMarketCap
api_key = '...'

# Tạo header cho yêu cầu API
headers = {
    'Accept': 'application/json',
    'X-CMC_PRO_API_KEY': api_key,
}


# 3. test Get tickers

In [3]:
#tickers = client.get_all_tickers()
#tickers

# 4. Binance

In [4]:
# Thư mục dataraw
data_folder = r"D:\HTP\Project-personal\CoinAnalytics\Main\data\raw"
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# Danh sách các token cần lấy và thời gian niêm yết
tokens = {
    'SANDUSDT': '2019-08-15',
    'AXSUSDT': '2020-10-17',
    'MANAUSDT': '2017-10-17',
    'GALAUSDT': '2021-09-15',
    'ENJUSDT': '2017-10-17',
    'SUSHIUSDT': '2020-09-01',
    'YGGUSDT': '2021-12-15',
    'ALICEUSDT': '2021-03-15'
}

In [5]:
def get_historical_data(symbol, start_date):
    klines = client.get_historical_klines(symbol, interval='1d', start_str=start_date)
    
    ohlcv_data = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 
                                                'close_time', 'quote_asset_volume', 'number_of_trades', 
                                                'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    
    ohlcv_data['timestamp'] = pd.to_datetime(ohlcv_data['timestamp'], unit='ms')

    order_book = client.get_order_book(symbol=symbol)
    highest_bid = order_book['bids'][0][0] if order_book['bids'] else None
    lowest_bid = order_book['bids'][-1][0] if order_book['bids'] else None
    highest_ask = order_book['asks'][0][0] if order_book['asks'] else None
    lowest_ask = order_book['asks'][-1][0] if order_book['asks'] else None
    
    ohlcv_data['highest_bid'] = highest_bid
    ohlcv_data['lowest_bid'] = lowest_bid
    ohlcv_data['highest_ask'] = highest_ask
    ohlcv_data['lowest_ask'] = lowest_ask
    
    return ohlcv_data

def save_data_to_csv(data, token):
    file_path = os.path.join(data_folder, f"{token}_historical_data.csv")
    
    if os.path.exists(file_path):
        os.remove(file_path)
    
    data.to_csv(file_path, index=False)
    print(f"Save data to {file_path} ... Done")


# 5. CMC

In [6]:
def get_token_info(symbol):
    # Chỉ sử dụng phần tên token (không có 'USDT')
    symbol = symbol.replace('USDT', '')
    
    url = "https://pro-api.coinmarketcap.com/v2/cryptocurrency/info"
    parameters = {'symbol': symbol}
    
    try:
        response = requests.get(url, headers=headers, params=parameters)
        response.raise_for_status()  # Kiểm tra lỗi HTTP
        data = response.json()

        if 'data' in data and symbol in data['data']:
            return data['data'][symbol][0]
        else:
            print(f"Không tìm thấy thông tin cho {symbol}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Lỗi khi gửi yêu cầu cho {symbol}: {e}")
        return None


# 6. Save

In [ ]:
def get_all_token_data():
    all_data = {}
    for token, date in tokens.items():
        print(f"History data for {token} niêm yết từ {date}")
        data = get_historical_data(token, date)
        all_data[token] = data
        save_data_to_csv(data, token)
        time.sleep(2)  # Dừng 2 giây giữa các lần gọi API để tránh bị rate limit
    return all_data

token_data = get_all_token_data()

In [ ]:
def save_token_info_to_json():
    """
    Lấy thông tin cho tất cả các token từ CoinMarketCap và lưu vào file JSON.
    """
    token_data = []
    for token in tokens.keys():
        print(f"\n Đang lấy info {token} ... ")
        info = get_token_info(token)

        if info:
            urls = info.get('urls', {})
            token_details = {
                "Token": token,
                "Description": info.get('description', 'Không có mô tả'),
                "Website": urls.get('website', ['Không có website'])[0] if urls.get('website') else 'Không có website',
                "Technical Doc": urls.get('technical_doc', ['Không có tài liệu kỹ thuật'])[0] if urls.get('technical_doc') else 'Không có tài liệu kỹ thuật',
                "Twitter": urls.get('twitter', ['Không có Twitter'])[0] if urls.get('twitter') else 'Không có Twitter',
                "Reddit": urls.get('reddit', ['Không có Reddit'])[0] if urls.get('reddit') else 'Không có Reddit',
                "Logo": info.get('logo', 'Không có logo')
            }
            token_data.append(token_details)
            print(f"{token} ... Done")
        else:
            print(f"không thể lấy info {token}.")
    
    output_file = os.path.join(data_folder, "token_info.json")
    try:
        with open(output_file, "w", encoding="utf-8") as json_file:
            json.dump(token_data, json_file, ensure_ascii=False, indent=4)
        print(f"\nSave data to '{output_file}' ... Done")
    except IOError as e:
        print(f"Lỗi khi ghi dữ liệu vào file '{output_file}': {e}")

save_token_info_to_json()